# PolyNER Demo Notebook

This notebook demonstrates the key features of the PolyNER library for multilingual Named Entity Recognition with emoji support.

## Setup

First, let's make sure that PolyNER is installed and we have all necessary dependencies.

In [1]:
# Uncomment to install polyner if not already installed
# !pip install polyner

# For development installation (if working from a local clone of the repo)
!pip install -e .

# We'll also need these packages for visualization
!pip install matplotlib seaborn

Obtaining file:///C:/Users/Yuuv/OneDrive/Desktop/Personal%20Projects/PolyNER
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   --------------------------------------- 590.6/590.6 kB 10.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=18246bd26c38b47087c02115c89c041624260c6c06d1ff2dbc3c6353075b1c36
  Stored in directory: c:\users\yuuv\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
  Running setup.py develop for polyner


  DEPRECATION: Legacy editable install of polyner==0.1.0 from file:///C:/Users/Yuuv/OneDrive/Desktop/Personal%20Projects/PolyNER (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [2]:
# Import PolyNER and related libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up matplotlib for better visualization
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

# Import PolyNER and related utilities
from polyner import PolyNER
from polyner.utils import (
    filter_by_language, 
    filter_emojis, 
    filter_by_entity,
    get_language_distribution, 
    get_entity_distribution,
    get_emoji_distribution
)
from polyner.entity_recognition import DictionaryEntityRecognizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yuuv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


## Feature 1: Basic Text Processing

Let's start with processing a simple English text to see the basic output format.

In [3]:
# Initialize the PolyNER processor
processor = PolyNER()

# Process a simple English text
text = "Apple Inc. is headquartered in Cupertino, California. The company was founded by Steve Jobs."
result = processor.process(text)

# Display the result
result

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Yuuv/nltk_data'
    - 'c:\\Users\\Yuuv\\anaconda3\\nltk_data'
    - 'c:\\Users\\Yuuv\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\Yuuv\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Yuuv\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### Understanding the Output

Let's examine the columns in the output DataFrame:

- `token`: The individual word or character
- `language`: Detected language code (e.g., 'en' for English)
- `is_emoji`: Boolean flag indicating if the token is an emoji
- `norm_token`: Normalized version of the token (lowercase, accents removed)
- `entity_label`: Named entity type if recognized (e.g., 'ORG' for organization, 'PERSON' for person)

Now, let's look at just the entities that were detected:

In [ ]:
# Filter to show only detected entities
entities = filter_by_entity(result)
entities

## Feature 2: Multilingual Support

Let's test PolyNER's ability to handle multiple languages in the same text.

In [ ]:
# Text with multiple languages
multilingual_text = """
Hello, my name is John Smith. I work for Microsoft in Seattle.
Bonjour, je m'appelle Pierre Dupont. Je travaille pour Google à Paris.
Hola, me llamo Carlos García. Trabajo para Amazon en Madrid.
你好，我叫李明。我在北京的百度工作。
"""

# Process the multilingual text
multi_result = processor.process(multilingual_text)

# Get language distribution
lang_dist = get_language_distribution(multi_result)

# Display the language distribution
print("Language distribution:")
for lang, count in lang_dist.items():
    print(f"{lang}: {count} tokens")

# Display a sample of each detected language
for lang in lang_dist.keys():
    print(f"\nSample of tokens in {lang}:")
    lang_tokens = filter_by_language(multi_result, lang)
    print(lang_tokens['token'].head().tolist())

In [ ]:
# Visualize language distribution
plt.figure(figsize=(10, 6))
plt.bar(lang_dist.keys(), lang_dist.values())
plt.title('Token Distribution by Language')
plt.xlabel('Language')
plt.ylabel('Number of Tokens')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Feature 3: Entity Recognition Across Languages

Let's see how well PolyNER recognizes entities in different languages.

In [ ]:
# Extract and display entities from the multilingual text
multi_entities = filter_by_entity(multi_result)
multi_entities

In [ ]:
# Get entity distribution by type
entity_dist = get_entity_distribution(multi_result)

# Visualize entity distribution
plt.figure(figsize=(10, 6))
plt.bar(entity_dist.keys(), entity_dist.values())
plt.title('Entity Distribution by Type')
plt.xlabel('Entity Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Feature 4: Emoji Handling

Let's test how PolyNER handles text with emojis.

In [ ]:
# Text with emojis
emoji_text = """
I love visiting New York City! 🗽 The Statue of Liberty is amazing. 😍
Tokyo 🗼 is also on my list, and I want to see the cherry blossoms! 🌸
Paris and the Eiffel Tower 🇫🇷 are romantic, especially at night. ✨
I'm saving up to go on a world tour! 🌍 ✈️ 🧳
"""

# Process the text
emoji_result = processor.process(emoji_text)

# Extract emojis
emojis = filter_emojis(emoji_result)
emojis

In [ ]:
# Get emoji distribution
emoji_dist = get_emoji_distribution(emoji_result)

# Display emoji distribution
print("Emoji distribution:")
for emoji, count in emoji_dist.items():
    print(f"{emoji}: {count}")

## Feature 5: Custom Entity Dictionaries

One of PolyNER's powerful features is the ability to add custom entity dictionaries for domain-specific recognition.

In [ ]:
# Create a custom dictionary recognizer
dict_recognizer = DictionaryEntityRecognizer()

# Add custom dictionaries for tech products and companies
dict_recognizer.add_entity_dictionary(
    "SMARTPHONE",
    ["iPhone 13", "iPhone 12", "Galaxy S21", "Pixel 6", "OnePlus 9"],
    case_sensitive=True
)

dict_recognizer.add_entity_dictionary(
    "LAPTOP",
    ["MacBook Pro", "MacBook Air", "Dell XPS", "ThinkPad X1", "Surface Laptop"],
    case_sensitive=True
)

dict_recognizer.add_entity_dictionary(
    "TECH_COMPANY",
    ["Apple", "Google", "Microsoft", "Amazon", "Meta", "Samsung", "Tesla"],
    case_sensitive=True
)

# Sample text about tech products
tech_text = """
I recently upgraded from my old iPhone 12 to the new iPhone 13 Pro Max, and the camera is amazing!
My friend uses a Galaxy S21 and loves it too. For work, I use a MacBook Pro, but I'm considering
switching to a Dell XPS or ThinkPad X1 for better compatibility with our office software.

Apple and Google are releasing new AI features, while Microsoft is focusing on cloud services.
Meta (formerly Facebook) is investing heavily in VR technology.
"""

# Recognize entities using the custom dictionaries
custom_entities = dict_recognizer.recognize_entities(tech_text)

# Display the results
print("Detected tech entities:")
for entity in custom_entities:
    print(f"{entity['text']} - {entity['label']}")

In [ ]:
# Count entities by type
entity_counts = {}
for entity in custom_entities:
    label = entity["label"]
    if label not in entity_counts:
        entity_counts[label] = 0
    entity_counts[label] += 1

# Display entity counts
print("\nEntity counts by type:")
for label, count in entity_counts.items():
    print(f"{label}: {count}")

# Visualize entity counts
plt.figure(figsize=(10, 6))
plt.bar(entity_counts.keys(), entity_counts.values())
plt.title('Custom Entity Distribution by Type')
plt.xlabel('Entity Type')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Feature 6: Combined Approach

Let's demonstrate how to combine the standard NER with custom dictionaries for a comprehensive approach.

In [ ]:
# Create a custom dictionary recognizer for medical terms
medical_recognizer = DictionaryEntityRecognizer()

# Add medical dictionaries
medical_recognizer.add_entity_dictionary(
    "MEDICATION",
    ["Aspirin", "Ibuprofen", "Paracetamol", "Amoxicillin", "Lipitor", "Prozac", "Metformin"],
    case_sensitive=True
)

medical_recognizer.add_entity_dictionary(
    "CONDITION",
    ["Hypertension", "Diabetes", "Type 2 Diabetes", "Asthma", "Arthritis", "Depression"],
    case_sensitive=True
)

# Sample medical text
medical_text = """
Patient John Smith (42) was admitted to Massachusetts General Hospital in Boston on June 15, 2023.
He has a history of Hypertension and Type 2 Diabetes. Current medications include Lipitor (20mg daily)
and Metformin (500mg twice daily). The patient reported taking Ibuprofen for occasional headaches.
Dr. Sarah Johnson recommended continuing current treatment and scheduled a follow-up appointment
in 3 months at the clinic in New York.
"""

# Process with standard NER
standard_result = processor.process(medical_text)

# Display standard NER results
print("Standard NER results:")
std_entities = standard_result[standard_result["entity_label"].notna()]
for _, row in std_entities.iterrows():
    print(f"{row['token']} - {row['entity_label']}")

# Process with custom medical dictionary
medical_entities = medical_recognizer.recognize_entities(medical_text)

# Display custom medical entities
print("\nCustom medical entities:")
for entity in medical_entities:
    print(f"{entity['text']} - {entity['label']}")

## Feature 7: Batch Processing

Let's demonstrate how to process multiple texts at once using the batch processing feature.

In [ ]:
# Create a batch of texts
texts = [
    "Apple Inc. is headquartered in Cupertino, California.",
    "Google's main office is in Mountain View.",
    "Amazon was founded by Jeff Bezos in Seattle.",
    "Microsoft is based in Redmond, Washington."
]

# Process the batch
batch_results = processor.process_batch(texts)

# Display entities from each text
for i, result in enumerate(batch_results):
    print(f"Text {i+1}: {texts[i]}")
    entities = result[result["entity_label"].notna()]
    if not entities.empty:
        print("Detected entities:")
        for _, row in entities.iterrows():
            print(f"  {row['token']} - {row['entity_label']}")
    else:
        print("No entities detected.")
    print()

## Feature 8: Analysis of Social Media Content

Social media content often contains a mix of languages, emojis, and entities. Let's see how PolyNER handles this type of content.

In [ ]:
# Sample social media post
social_media_post = """
Just had an amazing dinner at Le Petit Bistro in Paris! 🍷 The escargot was délicieux! 
Next stop: Berlin for Oktoberfest! 🍺 Can't wait to try some authentic German bratwurst.
私は日本食も大好きです。寿司と天ぷらは最高です！🍣
Follow my food adventures on Instagram @foodie_traveler and check out my latest YouTube video!
"""

# Process the social media post
social_result = processor.process(social_media_post)

# Display language distribution
social_lang_dist = get_language_distribution(social_result)
print("Language distribution:")
for lang, count in social_lang_dist.items():
    print(f"{lang}: {count} tokens")

# Display emojis
social_emojis = filter_emojis(social_result)
print("\nEmojis:")
print(social_emojis['token'].tolist())

# Display entities
social_entities = filter_by_entity(social_result)
print("\nDetected entities:")
for _, row in social_entities.iterrows():
    print(f"{row['token']} - {row['entity_label']}")

## Advanced Analysis: Visualizing the Results

Let's create a more comprehensive visualization of the analysis results.

In [ ]:
def analyze_and_visualize(text, title="Text Analysis"):
    # Process the text
    result = processor.process(text)
    
    # Get distributions
    lang_dist = get_language_distribution(result)
    entity_dist = get_entity_distribution(result)
    emoji_dist = get_emoji_distribution(result)
    
    # Create a figure with subplots
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(title, fontsize=16)
    
    # Plot language distribution
    if lang_dist:
        axs[0].bar(lang_dist.keys(), lang_dist.values())
        axs[0].set_title('Language Distribution')
        axs[0].set_xlabel('Language')
        axs[0].set_ylabel('Token Count')
        axs[0].tick_params(axis='x', rotation=45)
    else:
        axs[0].text(0.5, 0.5, 'No language data', ha='center', va='center')
        axs[0].set_title('Language Distribution')
    
    # Plot entity distribution
    if entity_dist:
        axs[1].bar(entity_dist.keys(), entity_dist.values(), color='green')
        axs[1].set_title('Entity Distribution')
        axs[1].set_xlabel('Entity Type')
        axs[1].set_ylabel('Count')
        axs[1].tick_params(axis='x', rotation=45)
    else:
        axs[1].text(0.5, 0.5, 'No entities detected', ha='center', va='center')
        axs[1].set_title('Entity Distribution')
    
    # Plot emoji distribution
    if emoji_dist:
        emojis = list(emoji_dist.keys())
        counts = list(emoji_dist.values())
        axs[2].bar(range(len(emojis)), counts, color='orange')
        axs[2].set_title('Emoji Distribution')
        axs[2].set_xlabel('Emoji')
        axs[2].set_ylabel('Count')
        axs[2].set_xticks(range(len(emojis)))
        axs[2].set_xticklabels(emojis, fontsize=14)
    else:
        axs[2].text(0.5, 0.5, 'No emojis detected', ha='center', va='center')
        axs[2].set_title('Emoji Distribution')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    
    # Return the processed result for further analysis
    return result

In [ ]:
# Analyze and visualize the social media post
social_result = analyze_and_visualize(social_media_post, "Social Media Post Analysis")

In [ ]:
# Analyze and visualize the news article
news_article = """
WASHINGTON (Reuters) - U.S. President Joe Biden met with Chinese President Xi Jinping 
at the G20 summit in Rome on Tuesday. The leaders discussed trade relations between 
the United States and China, as well as climate change initiatives ahead of the 
COP26 conference in Glasgow, Scotland.

Meanwhile, tech giants Apple and Google announced new AI partnerships, 
while Tesla's stock surged 5% after reporting record quarterly earnings.
"""

news_result = analyze_and_visualize(news_article, "News Article Analysis")

## Conclusion

This notebook has demonstrated the key features of the PolyNER library:

1. Basic text processing and entity recognition
2. Multilingual support
3. Emoji handling
4. Custom entity dictionaries
5. Combined approaches for domain-specific entity recognition
6. Batch processing
7. Analysis of mixed content (like social media posts)
8. Visualization of analysis results

PolyNER provides a comprehensive toolkit for processing and analyzing text that contains multiple languages, emojis, and specialized domain terminology.